In [46]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import serial
import tdwf
import time
import os

#-[Configurazione seriale]-----------------------------------------------------
ser = serial.Serial()
ser.port = "COM8"
ser.baudrate = 115200
ser.timeout = 1
ser.rts = False
ser.dtr = False
ser.open()

def query(message, flagR=True, flagW=True):
    if flagW:    
        message += '\n'
        ser.write(message.encode('utf-8'))
    if flagR:
        res = ser.readline().decode('utf-8')
        return res


for i in range(12):
    response = ser.readline().decode('utf-8')
    print(response[:-1])


#-[Configurazione AD2]-------------------------------------------------------
ad2 = tdwf.AD2(iconfig=2)

ad2.vdd = +5
ad2.power(True)

#   2. Configurazione generatore di funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.config(ampl = 5, offs= 0, func=tdwf.funcSine, freq = 40968)
wgen.w1.start()

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=5e5
scope.npt=int(10e6)
scope.ch1.rng = 10
scope.ch2.rng = 10
scope.trig(True, level = 1, sour = tdwf.trigsrcCh1, delay = 0.04)


#-[Saving pipeline]-----------------------------------------------------
Directory = f"dopplerWithAngle_{time.strftime('%Y%m%d_%H%M%S')}" # directory name based on the current date and time
parent_dir = "../../Data"
path = os.path.join(parent_dir, Directory) # creating the directory
os.mkdir(path)
print(f"Data will be saved in {path}\n")
log_file = os.path.join(path, f"log.txt")   #creating the log file
message = input("Please insert a \bMEANINGFUL\b comment to understand what the measurement in about: ")
with open(path+"/log.txt", "a") as log:
    log.write(time.strftime('%H%M%S') + ' >>> ' + message + "\n")
    log.close()















Dispositivo #1 [SN:210321B1F3F8, hdwf=1] connesso!
Configurazione #2
Data will be saved in ../../Data\dopplerWithAngle_20250515_112543



In [70]:
#-[Configurazione AD2]-------------------------------------------------------
ad2 = tdwf.AD2(iconfig=2)

ad2.vdd = +5
ad2.power(True)

#   2. Configurazione generatore di funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.config(ampl = 5, offs= 0, func=tdwf.funcSine, freq = 40968)
wgen.w1.start()

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=5e5
scope.npt=int(10e6)
scope.ch1.rng = 10
scope.ch2.rng = 10
scope.trig(True, level = 1, sour = tdwf.trigsrcCh1, delay = 0.04)

Dispositivo #1 [SN:210321B1F3F8, hdwf=1] connesso!
Configurazione #2


In [47]:


def setAngle(angle):
    print( query("ty="+str(angle))[:-1] )
    res = int(query("y?").split('=')[1])
    while res != angle:
        res = int(query("y?").split('=')[1])
    return res

def setX(x):
    print( query("tx="+str(x))[:-1] )
    res = int(query("x?").split('=')[1])
    while res != x:
        res = int(query("x?").split('=')[1])
    return res
















rangeX = [0, 80000]
rangeY = [0, 200]



#translations = np.linspace(rangeX[0], rangeX[1], 11, dtype=int)
angules = np.linspace(rangeY[0], rangeY[1], 4, dtype=int)

angules = np.delete(angules, np.where(angules == -400))
angules = np.delete(angules, np.where(angules == 400))
print(angules)

agulIndex = 0

[  0  66 133 200]


## MEASUREMENT ON THE RUN

In [71]:
tt, tx, rx = [], [], []



setX(0)
setAngle(angules[agulIndex])


posX = int(query("x?").split('=')[1])
first = True
time.sleep(.5)

res = query("tx=" + str(rangeX[1]))
print(res[:-1])
time.sleep(.5)
scope.sample()
while(posX < 0.98 * rangeX[1]):
    posX = int(query("x?").split('=')[1])
    print(posX)



tt = tt + scope.time.vals.tolist()
tx = tx + scope.ch1.vals.tolist()
rx = rx + scope.ch2.vals.tolist()
res = query("tx=" + str(rangeX[0]))
print(res[:-1])

info = f"Doppler Shift measurement - X = {rangeX[0]:5.0f} -> {rangeX[1]:5.0f} - vX = {query("sx?").split('=')[1] } - Y = {angules[agulIndex]} \n"

data_to_save = np.c_[tt, tx, rx]
info = info + f"time [s]\tch1 [V]\tch2 [V]\n"


filename = f"dopplerShift_{angules[agulIndex]}-{rangeX[0]:.0f}-{rangeX[1]:.0f}"

np.savetxt(path+'/'+filename+".txt", data_to_save, delimiter="\t", header=info, fmt="%s")

agulIndex += 1



TX=0
TY=200
TX=80000
80000
TX=0


In [33]:
res = query(input('>> '))
print(res[:-1])

TY=0


In [68]:
agulIndex

3

In [75]:
import matplotlib.pyplot as plt


plt.plot(tt[0: 100], tx[0:100], 'r', label='tx')
plt.plot(tt[0: 100], rx[0: 100], 'b', label='rx')
plt.show()

In [37]:
np.mean(np.diff(tt))

np.float64(1.2577853442633648e-08)

In [44]:
ser.close()

In [69]:
ad2.close()

Dispositivo disconnesso.
